In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create dataframe from boxing data csv 'bouts_out_new.csv'

df = pd.read_csv('bouts_out_new.csv')

# Create a dictionary with each numerical column name, and the acceptable range of values for that column.

num_cols_dict = {col: (df[col].quantile(0.01), df[col].quantile(0.99)) for col in df.describe().columns}

# Use the col_ranges dictionary to replace all unacceptable values with NaN, 
# and then fill the NaN values with the median of each column.

for col in num_cols_dict.keys():
    df[col] = df[col].where((df[col] >= num_cols_dict[col][0]) & (df[col] <= num_cols_dict[col][1]))
    df[col] = df[col].fillna(df[col].median())
    
# Remove the "stance" column, as there are too many null values. 
# Remove the "judge" columns, since the "result" column already denotes the winner.

df = df.drop(['stance_A','stance_B', 'judge1_A', 'judge1_B', 'judge2_A', 'judge2_B','judge3_A','judge3_B'], axis=1)



,age_A,age_B,height_A,height_B,reach_A,reach_B,weight_A,weight_B,won_A,won_B,lost_A,lost_B,drawn_A,drawn_B,kos_A,kos_B
count,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000
mean,24.291988,26.290553,172.858126,172.948475,178.241292,178.096256,141.859377,142.005278,21.630965,14.109541,4.976889,7.875525,2.498840,2.315412,9.659386,5.765223
std,4.393937,4.034855,6.709200,4.894657,5.657018,3.387027,16.626665,16.378102,23.736692,17.098036,7.243395,9.080735,4.458731,3.869040,10.470511,7.364659
min,16.000000,17.000000,155.000000,155.000000,157.000000,155.000000,105.000000,105.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21.000000,24.000000,170.000000,173.000000,178.000000,178.000000,140.000000,140.000000,5.000000,2.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000
50%,24.000000,26.000000,173.000000,173.000000,178.000000,178.000000,140.000000,140.000000,13.000000,8.000000,2.000000,5.000000,1.000000,1.000000,6.000000,3.000000
75%,27.000000,27.000000,175.000000,173.000000,178.000000,178.000000,140.000000,140.000000,30.000000,20.000000,7.000000,11.000000,3.000000,3.000000,14.000000,8.000000
max,39.000000,41.000000,196.000000,195.000000,211.000000,210.000000,247.000000,248.000000,129.000000,93.000000,41.000000,52.000000,27.000000,24.000000,57.000000,39.000000


In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df.describe()


,age_A,age_B,height_A,height_B,reach_A,reach_B,weight_A,weight_B,won_A,won_B,lost_A,lost_B,drawn_A,drawn_B,kos_A,kos_B
count,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000,387427.000000
mean,24.291988,26.290553,172.858126,172.948475,178.241292,178.096256,141.859377,142.005278,21.630965,14.109541,4.976889,7.875525,2.498840,2.315412,9.659386,5.765223
std,4.393937,4.034855,6.709200,4.894657,5.657018,3.387027,16.626665,16.378102,23.736692,17.098036,7.243395,9.080735,4.458731,3.869040,10.470511,7.364659
min,16.000000,17.000000,155.000000,155.000000,157.000000,155.000000,105.000000,105.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21.000000,24.000000,170.000000,173.000000,178.000000,178.000000,140.000000,140.000000,5.000000,2.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000
50%,24.000000,26.000000,173.000000,173.000000,178.000000,178.000000,140.000000,140.000000,13.000000,8.000000,2.000000,5.000000,1.000000,1.000000,6.000000,3.000000
75%,27.000000,27.000000,175.000000,173.000000,178.000000,178.000000,140.000000,140.000000,30.000000,20.000000,7.000000,11.000000,3.000000,3.000000,14.000000,8.000000
max,39.000000,41.000000,196.000000,195.000000,211.000000,210.000000,247.000000,248.000000,129.000000,93.000000,41.000000,52.000000,27.000000,24.000000,57.000000,39.000000


In [ ]:
col_ranges = {'age_A': (16,39), 'age_B': (16,39),
        'height_A': (155, 196), 'height_B': (155,196),
        'reach_A' : (157,211), 'reach_B' : (157,211),
        'weight_A': (147,210), 'weight_B': (147,210),
        'won_A': (0, 150), 'won_B': (0, 150),
        'drawn_A': (0,50), 'drawn_B': (0,50)}